### Introduction
The LegalRuleML Core specification is a vaste repository of normative and non-normative ontologies for the XML file format generation. This also contains examples that can be utilised to feed Llama along with a system prompt. The model used is Llama2-7b-chat.

### ToC

In [**Approach 1**](#Approach1), the few shot example given is a previously generated XML file.
In [**Approach 2**](#Approach2), the examples given in Section 6 of the Core specification are fed to Llama2.

### Results

While in the first approach, the model ignored the fact that Constitutive Rules and associations may be present, the better quality of the prompts gave a significant improvement. However, the issue of rationalising the separately fed prompts across different sections of the statute will require perhaps a **larger context window** or **XML tweaking**

In [14]:
import urllib.request
import json
import os
import ssl



def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script




def send_request(section_for_conversion):

    data = {
      "messages": [
        {
          "role": "system",
          "content": "You are a consistent Legal Rule ML converter who takes the statute text as input and outputs the XML format for the same"
        },
        {
          "role": "user",
          "content": "Convert this statute to Legal Rule ML XML format: Punishment for piracy - Whoever commits any act of piracy shall be punished with imprisonment for life or with fine or with both; or with death if the act of piracy causes death."
        },
        {
          "role": "assistant",
          "content": """<LegalRuleML xmlns:lrml="http://www.oasis-open.org/committees/legalruleml">
      <lrml:PrescriptiveStatement id="PunishmentForPiracy">
        <lrml:Rule id="RuleForPiracyPunishment">
          <lrml:if>
          <lrml:Fact>
              <lrml:Rel iri="#commitPiracy"/>
            </lrml:Fact>
          </lrml:if>
          <lrml:then>
            <lrml:Obligation>
              <lrml:Sanction>
                <lrml:Penalty>
                  <lrml:Type iri="#imprisonment"/>
                  <lrml:Duration max="life"/>
                </lrml:Penalty>
                <lrml:Condition>
                  <lrml:CausesDeathOrAttempt/>
                  <lrml:AdditionalSanctions>
                    <lrml:Restitution/>
                    <lrml:ForfeitureOfProperty/>
                  </lrml:AdditionalSanctions>
                </lrml:Condition>
              </lrml:Sanction>
            </lrml:Obligation>
          </lrml:then>
        </lrml:Rule>
      </lrml:PrescriptiveStatement>
    </LegalRuleML>
    """
        },
        {
          "role": "user",
          "content": f"""Convert this extract of a statute to LegalRuleML XML format: {section_for_conversion}"""
        }
      ],
      "temperature": 0.25,
      "max_tokens": 500
    }

    
    body = str.encode(json.dumps(data))
    
    
    url = ''
    # Replace this with the primary/secondary key or AMLToken for the endpoint
    api_key = ''
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")
    
    
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
    
    req = urllib.request.Request(url, body, headers)
    
    try:
        response = urllib.request.urlopen(req)
    
        result = response.read()
        #print(result)

        return result
        
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))




In [15]:
import pandas as pd

def csv_parser():

    csv_filename = 'sections.csv' # file containing all the sections from the E-Waste Rules, generated via IK_interface function

    df_condition = "1"
    # Read the CSV file into a Pandas DataFrame
    df = pd.read_csv(csv_filename)

    if df_condition == "1":
        df2 = df.head(10)
    else:
        df2=df.copy()

    return df2

    

In [16]:
print(csv_parser())

                              Section Title  \
0  1. Short title, extent and commencement.   
1  1. Short title, extent and commencement.   
2  1. Short title, extent and commencement.   
3                           2. Definitions.   
4                           2. Definitions.   
5                           2. Definitions.   
6                           2. Definitions.   
7                           2. Definitions.   
8                           2. Definitions.   
9                           2. Definitions.   

                                      Nested Content  
0  This Act may be called the Surrogacy (Regulati...  
1  This Act may be called the Surrogacy (Regulati...  
2  It shall come into force on such date1 as the ...  
3  In this Act, unless the context otherwise requ...  
4  In this Act, unless the context otherwise requ...  
5  âabandoned childâ means a child born out o...  
6  âaltruistic surrogacyâ means the surrogacy...  
7  âappropriate authorityâ means the a

In [18]:

# Now, 'df' contains the data from the CSV file
    # You can perform various operations on the DataFrame
section_for_conversion = ''

combined_response = ''

df2 = csv_parser()
# Iterate through rows and process the data
for index, row in df2.iterrows():
    section_title = row['Section Title']
    nested_content = row['Nested Content']

    # Process section_title and nested_content as needed
    #print(f"Section Title: {section_title}")
    #print(f"Nested Content: {nested_content}\n")
    section_for_conversion = f"{section_title}: {nested_content}\n"

    result=send_request(section_for_conversion)
    
    result_json = json.loads(result.decode('utf-8'))
    
    # Extract the value inside the "text" key
    text_value = result_json['choices'][0]['message']['content']

    combined_response += text_value


In [19]:
print (combined_response)

Here is the LegalRuleML XML format for the extract of the statute you provided:
```
<LegalRuleML xmlns:lrml="http://www.oasis-open.org/committees/legalruleml">
  <lrml:LegalDocument id="SurrogacyRegulationAct2021">
    <lrml:ShortTitle>Surrogacy (Regulation) Act, 2021</lrml:ShortTitle>
    <lrml:Extent>
      <lrml:Date start="2021-01-01" end="2021-12-31"/>
    </lrml:Extent>
    <lrml:Citation>
      <lrml:Reference iri="http://www.legislation.gov.in/sites/default/files/Acts/A2021-15.pdf"/>
    </lrml:Citation>
    <lrml:Commencement>
      <lrml:Date start="2021-01-01" end="2021-12-31"/>
      <lrml:Condition>
        <lrml:Notification iri="http://www.egazette.nic.in/pdf/2021/Egazette-2021-1-15.pdf"/>
      </lrml:Condition>
    </lrml:Commencement>
  </lrml:LegalDocument>
</LegalRuleML>
```
Explanation:

* `lrml:LegalDocument` is the root element of the LegalRuleML document, which represents the statute.
* `lrml:ShortTitle` contains the short title of the statute, which is "Surroga

## Approach2

Using the examples shown in the Documentation as few-shot examples

In order to avoid oversight of the nuances of LegalRuleML representation which involves associations, defeasibility, and different kinds of normative rules, two examples from Section 6 in the documentation available [here](http://docs.oasis-open.org/legalruleml/legalruleml-core-spec/v1.0/legalruleml-core-spec-v1.0.html) is attempted to be passed as a few-shot prompt here.

In [22]:
import urllib.request
import json
import os
import ssl



def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script




def send_request(section_for_conversion):

    data = {
      "messages": [
        {
          "role": "system",
          "content": "You are a consistent Legal Rule ML converter who takes the statute text as input and outputs the XML format for the same with reference to the documentation available at http://docs.oasis-open.org/legalruleml/legalruleml-core-spec/v1.0/legalruleml-core-spec-v1.0.html"
        },
        {
          "role": "user",
          "content": "Convert this statute extract to Legal Rule ML XML format: The benefit referred to in comma 1 shall be paid in an amount equal 80 per cent of five-twelfths of the income earned and reported for tax purposes by the freelancer in the second year preceding the year of application."
        },
        {
          "role": "assistant",
          "content": """
          <lrml:ConstitutiveStatement key="tax2">

          <ruleml:Rule closure="universal">

        <lrml:Paraphrase>If income is reported in some year, then it was earned in the previous year.

        </lrml:Paraphrase>

        <ruleml:if>

          <ruleml:Atom>

            <ruleml:Rel iri=":reported"/>

            <ruleml:Var>income</ruleml:Var>

            <ruleml:Var>year</ruleml:Var>

          </ruleml:Atom>

        </ruleml:if>

        <ruleml:then>

          <ruleml:Atom>

            <ruleml:Rel iri=":earned"/>

            <ruleml:Var>income</ruleml:Var>

            <ruleml:Expr key=":year-1">

              <ruleml:Fun iri=":subtract"/>

              <ruleml:Var>year</ruleml:Var>

              <ruleml:Data xsi:type="xs:integer">1</ruleml:Data>

            </ruleml:Expr>

          </ruleml:Atom>

        </ruleml:then>

      </ruleml:Rule>

    </lrml:ConstitutiveStatement>
    <lrml:PrescriptiveStatement key="literal">

    <ruleml:Rule closure="universal" key=":literal-template">

        <ruleml:if>

            <ruleml:And>

                <ruleml:Atom key=":atom-earned">

                    <ruleml:Rel iri=":earned"/>

                     <ruleml:Var>income</ruleml:Var>

                     <ruleml:Expr>

                        <ruleml:Fun iri=":subtract"/>

                        <ruleml:Var>year</ruleml:Var>

                        <ruleml:Data xsi:type="xs:integer">2</ruleml:Data>

                     </ruleml:Expr>

                 </ruleml:Atom>

                 <ruleml:Atom key=":atom-reported">

                     <ruleml:Rel iri=":reported"/>

                     <ruleml:Var>income</ruleml:Var>

                    <ruleml:Expr>

                         <ruleml:Fun iri=":subtract"/>

                         <ruleml:Var>year</ruleml:Var>

                         <ruleml:Data xsi:type="xs:integer">2</ruleml:Data>

                     </ruleml:Expr>

                 </ruleml:Atom>

            </ruleml:And>

        </ruleml:if>

        <ruleml:then>

             <lrml:Obligation key="obl-paybenefit">

                  <ruleml:slot>

                      <lrml:Bearer/>

                      <ruleml:Var>Employer</ruleml:Var>

                  </ruleml:slot>

                  <ruleml:slot>

                      <lrml:AuxiliaryParty/>

                      <ruleml:Var>Freelancer</ruleml:Var>

                  </ruleml:slot>

                  <ruleml:Atom>

                      <ruleml:Rel iri=":paybenefit"/>

                     <ruleml:Expr>

                          <ruleml:Fun iri=":80_percent_of_five-twelfths_of"/>

                          <ruleml:Var>income</ruleml:Var>

                      </ruleml:Expr>

                      <ruleml:Var>year</ruleml:Var>

                  </ruleml:Atom>

              </lrml:Obligation>

          </ruleml:then>

      </ruleml:Rule>

</lrml:PrescriptiveStatement>
    """
        },
        {
          "role": "user",
          "content": "Convert this statute extract to Legal Rule ML XML format:A person must not engage in a credit activity if the person does not hold a licence authorising the person to engage in the credit activity. Civil penalty: 2,000 penalty units.  Criminal penalty: 200 penalty units, or 2 years imprisonment, or both."
        },
        {
          "role": "assistant",
          "content": """

<lrml:LegalSources>

   <lrml:LegalSource key="ls1"             sameAs="http://www.comlaw.gov.au/Details/C2009A00134/Html/Text#param43"/>

</lrml:LegalSources>


<lrml:Context key="psInfo1">

   <lrml:appliesAssociations>

          <lrml:Associations>

                <lrml:Association>

                       <lrml:appliesSource keyref="#ls1/>

                       <lrml:toTarget keyref="#ps1"/>

                       <lrml:toTarget keyref="#ps2"/>

                       <lrml:toTarget keyref="#pen1"/>

                       <lrml:toTarget keyref="#pen2"/>

                </lrml:Association>

          </lrml:Associations>

</lrml:appliesAssociations>

</lrml:Context>


<lrml:Statements key="textblock1">

<lrml:hasQualification>

  <lrml:Override over="#ps2" under="#ps1"/>

  </lrml:hasQualification>

  <lrml:PrescriptiveStatement key="ps1">

      <ruleml:Rule key=":rule1" closure="universal">

        <lrml:hasStrength>

          <lrml:DefeasibleStrength key="str1" iri="&defeasible-ontology;#defeasible1"/>

        </lrml:hasStrength>

        <ruleml:if>

          <ruleml:Atom>

            <ruleml:Rel iri=":person"/>

            <ruleml:Var>X</ruleml:Var>

          </ruleml:Atom>

        </ruleml:if>

        <ruleml:then>

          <lrml:SuborderList>

            <lrml:Prohibition>

              <ruleml:Atom>

                <ruleml:Rel iri=":engageCreditActivity"/>

                <ruleml:Var>X</ruleml:Var>

              </ruleml:Atom>

            </lrml:Prohibition>

          </lrml:SuborderList>

        </ruleml:then>

      </ruleml:Rule>

    </lrml:PrescriptiveStatement>

 

    <lrml:PenaltyStatement key="pen2">

      <lrml:SuborderList>

        <lrml:Obligation>

          <ruleml:Atom>

            <ruleml:Rel iri=":payPenalUnits"/>

            <ruleml:Var>X</ruleml:Var>

            <ruleml:Ind>200</ruleml:Ind>

          </ruleml:Atom>

        </lrml:Obligation>

        <lrml:Obligation>

          <ruleml:Atom>

            <ruleml:Rel iri=":imprisonment"/>

            <ruleml:Var>X</ruleml:Var>

            <ruleml:Ind>2 years</ruleml:Ind>

          </ruleml:Atom>

        </lrml:Obligation>

        <lrml:Obligation>

          <ruleml:Atom>

            <ruleml:Rel iri=":payPenalUnitPlusImprisonment"/>

            <ruleml:Var>X</ruleml:Var>

            <ruleml:Ind>200</ruleml:Ind>

            <ruleml:Ind>2 years</ruleml:Ind>

          </ruleml:Atom>

        </lrml:Obligation>

      </lrml:SuborderList>

    </lrml:PenaltyStatement>

 

    <lrml:ReparationStatement key="rep1">

      <lrml:Reparation key="assoc1">

        <lrml:appliesPenalty keyref="#pen1"/>

        <lrml:toPrescriptiveStatement keyref="#ps1"/>

      </lrml:Reparation>

    </lrml:ReparationStatement>

    <lrml:ReparationStatement key="rep2">

      <lrml:Reparation keyref="#assoc1">

        <lrml:appliesPenalty keyref="#pen2"/>

        <lrml:toPrescriptiveStatement keyref="#ps1"/>

      </lrml:Reparation>

    </lrml:ReparationStatement>

</lrml:Statements>
"""
        },
        {
          "role": "user",
          "content": f"""Convert this extract of a statute to LegalRuleML XML format: {section_for_conversion}"""
        }
      ],
      "temperature": 0.25,
      "max_tokens": 500
    }

    
    body = str.encode(json.dumps(data))
    
    
    url = ''
    # Replace this with the primary/secondary key or AMLToken for the endpoint
    api_key = ''
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")
    
    
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
    
    req = urllib.request.Request(url, body, headers)
    
    try:
        response = urllib.request.urlopen(req)
    
        result = response.read()
        #print(result)

        return result
        
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))




In [25]:
import pandas as pd

def csv_parser():

    csv_filename = 'sections.csv' # file containing all the sections from the E-Waste Rules, generated via IK_interface function

    df_condition = "1"
    # Read the CSV file into a Pandas DataFrame
    df = pd.read_csv(csv_filename)

    if df_condition == "1":
        df2 = df.iloc[70:85]
    else:
        df2=df.copy()

    return df2

    

In [27]:
print(csv_parser())

                                      Section Title  \
70           11. Registration of surrogacy clinics.   
71           11. Registration of surrogacy clinics.   
72           11. Registration of surrogacy clinics.   
73           11. Registration of surrogacy clinics.   
74                 12. Certificate of registration.   
75                 12. Certificate of registration.   
76                 12. Certificate of registration.   
77                 12. Certificate of registration.   
78                 12. Certificate of registration.   
79  13. Cancellation or suspension of registration.   
80  13. Cancellation or suspension of registration.   
81  13. Cancellation or suspension of registration.   
82  13. Cancellation or suspension of registration.   
83                                      14. Appeal.   
84                                      14. Appeal.   

                                       Nested Content  
70  No person shall establish any surrogacy clinic...  
71  Eve

In [28]:

# Now, 'df' contains the data from the CSV file
    # You can perform various operations on the DataFrame
section_for_conversion = ''

combined_response = ''

df2 = csv_parser()
# Iterate through rows and process the data
for index, row in df2.iterrows():
    section_title = row['Section Title']
    nested_content = row['Nested Content']

    # Process section_title and nested_content as needed
    #print(f"Section Title: {section_title}")
    #print(f"Nested Content: {nested_content}\n")
    section_for_conversion = f"{section_title}: {nested_content}\n"

    result=send_request(section_for_conversion)
    
    result_json = json.loads(result.decode('utf-8'))
    
    # Extract the value inside the "text" key
    text_value = result_json['choices'][0]['message']['content']

    combined_response += text_value


In [29]:
print (combined_response)

Here is the statute extract in LegalRuleML XML format:
```
<lrml:LegalSources>
  <lrml:LegalSource key="ls1" sameAs="http://www.example.com/surrogacy-act"/>
</lrml:LegalSources>

<lrml:Context key="psInfo1">
  <lrml:appliesAssociations>
    <lrml:Associations>
      <lrml:Association keyref="#ls1">
        <lrml:toTarget keyref="#ps1"/>
      </lrml:Association>
    </lrml:Associations>
  </lrml:appliesAssociations>
</lrml:Context>

<lrml:Statements key="textblock1">
  <lrml:hasQualification>
    <lrml:Override over="#ps2" under="#ps1"/>
  </lrml:hasQualification>
  <lrml:PrescriptiveStatement key="ps1">
    <ruleml:Rule key=":rule1" closure="universal">
      <lrml:hasStrength>
        <lrml:DefeasibleStrength key="str1" iri="&defeasible-ontology;#defeasible1"/>
      </lrml:hasStrength>
      <ruleml:if>
        <ruleml:Atom>
          <ruleml:Rel iri=":establishSurrogacyClinic"/>
          <ruleml:Var>X</ruleml:Var>
        </ruleml:Atom>
      </ruleml:if>
      <ruleml:then>
     